## Introduction
In this notebook we'll explore some of the data in the CIA World Factbook

In [1]:
!conda install -yc conda-forge ipython-sql

Solving environment: done

## Package Plan ##

  environment location: /Users/bikram/opt/anaconda3

  added / updated specs:
    - ipython-sql


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-22.9.0               |   py39h6e9494a_2         962 KB  conda-forge
    ipython-sql-0.3.9          |  pyhd8ed1ab_1004          18 KB  conda-forge
    prettytable-3.4.1          |     pyhd8ed1ab_0          27 KB  conda-forge
    python_abi-3.9             |           2_cp39           4 KB  conda-forge
    sqlparse-0.4.3             |     pyhd8ed1ab_0          35 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.0 MB

The following NEW packages will be INSTALLED:

  ipython-sql        conda-forge/noarch::ipython-sql-0.3.9-pyhd8ed1ab_1004 None
  prettytable        conda-forge/noarch::prettytable-3.4.1-pyhd

In [2]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [3]:
%%sql
SELECT *
    FROM sqlite_master
WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [5]:
%%sql
SELECT *
    FROM facts
LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:
1. name - the name of the country
2. area - the country's total area (both land and water)
3. area_land - the country's land in square kilometers
4. area_water - the country's waterarea in square kilometers
5. population - the country's population
6. population_growth - the country's population growth as a percentage
7. birth_rate - the country's birth rate, the number of births per year per 1,000 people
8. death_rate - the country's death rate, the number of deaths per year per 1,000 people

In [7]:
%%sql
SELECT MIN(population) AS "Minimum Population", MAX(population) AS "Maximum Population", 
MIN(population_growth) AS "Minimum Population Growth", MAX(population_growth) AS "Maximum Population Growth"
    FROM facts

 * sqlite:///factbook.db
Done.


Minimum Population,Maximum Population,Minimum Population Growth,Maximum Population Growth
0,7256490011,0.0,4.02


We can see that there's a country with a population of 0 and there's a country with a population with over 7 billion. 

In [8]:
%%sql 
SELECT name
    FROM facts
WHERE population = 0

 * sqlite:///factbook.db
Done.


name
Antarctica


It seems like the country with the population of 0 was Antartica.

In [9]:
%%sql 
SELECT name
    FROM facts
WHERE population = (SELECT MAX(population) FROM facts)

 * sqlite:///factbook.db
Done.


name
World


We also see that the table contains a row for the whole world which explains the maximum population of 7 billion.
Now that we know this, we should recalculate the summary statistics without this row. 

In [11]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
    FROM facts
WHERE name <> "World"

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


We can see that after excluding the `World` row, the max population for a country is a little over 1 billion.

In [18]:
%%sql
SELECT name
    FROM facts
WHERE population = (SELECT MAX(population) FROM facts WHERE name <> "World")

 * sqlite:///factbook.db
Done.


name
China


The country with the largest population is `China`.

In [13]:
%%sql
SELECT ROUND(AVG(population), 2) AS "Average Population", ROUND(AVG(area),2) AS "Average Area"
    FROM facts
WHERE name <> "World"

 * sqlite:///factbook.db
Done.


Average Population,Average Area
32242666.57,555093.55


## Finding Densely Populated Countries
We'll build a query to find countries that are densely populated. We'll indentify countries that have:    
1. Above average values for population
2. Below average values for area

In [19]:
%%sql
SELECT *
    FROM facts
WHERE population > (SELECT AVG(population) FROM facts WHERE name <> "World") AND area < (SELECT AVG(area) FROM facts WHERE name <> "World")

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


Some of these countries are known to be densely populated so we can have confidence in our results.

## Finding countries that have more water than land
We'll build a query to find the countries that have more water than land.

In [20]:
%%sql
SELECT *
    FROM facts
WHERE area_water > area_land

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67


We can see that there are only two countries that have more land than water.

## Countries with high death rates
We'll build queries to see: 
1. Which countries have the highest death rates?
2. Which countries have a higher death rate than birth rate?

In [23]:
%%sql
SELECT *
    FROM facts
ORDER BY death_rate DESC
LIMIT 10

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
98,lt,Lesotho,30355,30355,0,1947701,0.32,25.47,14.89,7.36
183,up,Ukraine,603550,579330,24220,44429471,0.6,10.72,14.46,2.25
26,bu,Bulgaria,110879,108489,2390,7186893,0.58,8.92,14.44,0.29
71,pu,Guinea-Bissau,36125,28120,8005,1726170,1.91,33.38,14.33,0.0
96,lg,Latvia,64589,62249,2340,1986705,1.06,10.0,14.31,6.26
35,cd,Chad,None,1259200,24800,11631456,1.89,36.6,14.28,3.45
102,lh,Lithuania,65300,62680,2620,2884433,1.04,10.1,14.27,6.27
122,wa,Namibia,824292,823290,1002,2212307,0.59,19.8,13.91,0.0
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
34,ct,Central African Republic,622984,622984,0,5391539,2.13,35.08,13.8,0.0


In [24]:
%%sql
SELECT *
    FROM facts
WHERE death_rate > birth_rate
ORDER BY death_rate DESC

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
183,up,Ukraine,603550,579330,24220,44429471,0.6,10.72,14.46,2.25
26,bu,Bulgaria,110879,108489,2390,7186893,0.58,8.92,14.44,0.29
96,lg,Latvia,64589,62249,2340,1986705,1.06,10.0,14.31,6.26
102,lh,Lithuania,65300,62680,2620,2884433,1.04,10.1,14.27,6.27
143,rs,Russia,17098242,16377742,720500,142423773,0.04,11.6,13.69,1.69
153,ri,Serbia,77474,77474,0,7176794,0.46,9.08,13.66,0.0
16,bo,Belarus,207600,202900,4700,9589689,0.2,10.7,13.36,0.7
75,hu,Hungary,93028,89608,3420,9897541,0.22,9.16,12.73,1.33
116,md,Moldova,33851,32891,960,3546847,1.03,12.0,12.59,9.67
57,en,Estonia,45228,42388,2840,1265420,0.55,10.51,12.4,3.6


We can see that there are `24` countries that have a higher death rate than birth rate